<a href="https://colab.research.google.com/github/fd-journey/journey/blob/main/Workshop_SQL_Part_2_Jeremy_Bowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Frederic Daoud** frederic.daoud@gmail.com

#NOTE:
Please insert a text cell at the top of the Notebook with your Full Name in Bold and your email that you use for both Journey and Github

Also, add to the title of the notebook, an underscore as well as the name of the killer from the SQL Murder Mystery activity.

Ex. 'Workshop_SQL_Part_2_Killer'

#Warmup

Run the following lines in the cell below:


```
import pandas as pd
import sqlite3 as sql
conn = sql.connect('example.db')
```

**Note:** You can run a cell by pressing 'Shift+Enter'

In [1]:
import pandas as pd
import sqlite3 as sql
conn = sql.connect('example.db')

#Covid-19 Reporting

We are going to be working with the Covid-19 Data set again, only we are going to be converting the dataframe to a sql table. In the line below, read in the url into a data frame, convert it to a sql table and show all columns in this table but limit it to the first 5 rows.

In [2]:
url = 'https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/e5bf35bc-e681-43da-b2ce-0242d00922ad?format=csv'
df = pd.read_csv(url)
df.to_sql("covid", con = conn)
pd.read_sql("SELECT * FROM covid LIMIT 5", con=conn)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


,index,_id,Assigned_ID,Outbreak Associated,Age Group,Neighbourhood Name,FSA,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Outcome,Currently Hospitalized,Currently in ICU,Currently Intubated,Ever Hospitalized,Ever in ICU,Ever Intubated
0,0,174440,1,Sporadic,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-22,2020-01-23,FEMALE,RESOLVED,No,No,No,No,No,No
1,1,174441,2,Sporadic,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-21,2020-01-23,MALE,RESOLVED,No,No,No,Yes,No,No
2,2,174442,3,Sporadic,20 to 29 Years,Parkwoods-Donalda,M3A,Travel,CONFIRMED,2020-02-05,2020-02-21,FEMALE,RESOLVED,No,No,No,No,No,No
3,3,174443,4,Sporadic,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-16,2020-02-25,FEMALE,RESOLVED,No,No,No,No,No,No
4,4,174444,5,Sporadic,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-20,2020-02-26,MALE,RESOLVED,No,No,No,No,No,No


In the cell below, select the 'Neighbourhood Name' column. 

In [3]:
pd.read_sql(
"SELECT [Neighbourhood Name] \
FROM covid"
,con = conn
)

,Neighbourhood Name
0,Willowdale East
1,Willowdale East
2,Parkwoods-Donalda
3,Church-Yonge Corridor
4,Church-Yonge Corridor
...,...
93180,Rouge
93181,North Riverdale
93182,Rouge
93183,Thistletown-Beaumond Heights


Select the first 10 rows in the table where the infected patient was female. Then call the first 15 rows in the dataframe where the patient was male, ever hospitalized and in their 40s.

In [4]:
pd.read_sql(
    "SELECT * \
    FROM covid \
    WHERE [Client Gender] = 'FEMALE' \
    AND [Ever Hospitalized] = 'Yes' \
    LIMIT 10"
    ,con = conn
)

,index,_id,Assigned_ID,Outbreak Associated,Age Group,Neighbourhood Name,FSA,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Outcome,Currently Hospitalized,Currently in ICU,Currently Intubated,Ever Hospitalized,Ever in ICU,Ever Intubated
0,21,174461,23,Sporadic,60 to 69 Years,Malvern,M1B,Travel,CONFIRMED,2020-03-04,2020-03-08,FEMALE,RESOLVED,No,No,No,Yes,Yes,Yes
1,34,174474,38,Sporadic,19 and younger,Dovercourt-Wallace Emerson-Junction,M6H,Travel,CONFIRMED,2020-03-09,2020-03-11,FEMALE,RESOLVED,No,No,No,Yes,No,No
2,94,174534,98,Sporadic,60 to 69 Years,Bayview Woods-Steeles,M2K,Close Contact,CONFIRMED,2020-03-10,2020-03-16,FEMALE,RESOLVED,No,No,No,Yes,Yes,Yes
3,128,174568,132,Sporadic,70 to 79 Years,Milliken,M1V,Travel,CONFIRMED,2020-03-12,2020-03-17,FEMALE,RESOLVED,No,No,No,Yes,Yes,Yes
4,192,174632,201,Outbreak Associated,60 to 69 Years,Morningside,M1E,"Outbreaks, Healthcare Institutions",CONFIRMED,2020-03-18,2020-03-19,FEMALE,RESOLVED,No,No,No,Yes,No,No
5,256,174696,271,Sporadic,30 to 39 Years,Lawrence Park South,M4N,Community,CONFIRMED,2020-03-17,2020-03-21,FEMALE,RESOLVED,No,No,No,Yes,No,No
6,275,174715,291,Sporadic,80 to 89 Years,Princess-Rosethorn,M9B,Community,CONFIRMED,2020-03-15,2020-03-21,FEMALE,RESOLVED,No,No,No,Yes,No,No
7,279,174719,296,Sporadic,70 to 79 Years,Black Creek,M3N,Close Contact,CONFIRMED,2020-03-13,2020-03-22,FEMALE,RESOLVED,No,No,No,Yes,Yes,Yes
8,359,174799,382,Sporadic,60 to 69 Years,Leaside-Bennington,M4G,Close Contact,CONFIRMED,2020-03-17,2020-03-24,FEMALE,RESOLVED,No,No,No,Yes,Yes,Yes
9,371,174811,396,Sporadic,50 to 59 Years,Victoria Village,M4A,Community,CONFIRMED,2020-03-14,2020-03-24,FEMALE,RESOLVED,No,No,No,Yes,Yes,Yes


In [5]:
pd.read_sql(
    "SELECT * \
    FROM covid \
    WHERE [Client Gender] = 'MALE' \
    AND [Age Group] = '60 to 69 Years' \
    AND [Ever Hospitalized] = 'Yes' \
    LIMIT 15"
    ,con = conn
)

,index,_id,Assigned_ID,Outbreak Associated,Age Group,Neighbourhood Name,FSA,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Outcome,Currently Hospitalized,Currently in ICU,Currently Intubated,Ever Hospitalized,Ever in ICU,Ever Intubated
0,7,174447,8,Sporadic,60 to 69 Years,Willowdale West,M2N,Travel,CONFIRMED,2020-02-21,2020-03-04,MALE,RESOLVED,No,No,No,Yes,No,No
1,35,174475,39,Sporadic,60 to 69 Years,Mount Pleasant West,M4S,Travel,CONFIRMED,2020-03-08,2020-03-12,MALE,RESOLVED,No,No,No,Yes,No,No
2,57,174497,61,Sporadic,60 to 69 Years,Agincourt South-Malvern West,M1S,Travel,CONFIRMED,2020-03-04,2020-03-12,MALE,RESOLVED,No,No,No,Yes,Yes,Yes
3,100,174540,104,Sporadic,60 to 69 Years,Bay Street Corridor,M5S,Travel,CONFIRMED,2020-03-08,2020-03-15,MALE,RESOLVED,No,No,No,Yes,Yes,No
4,250,174690,265,Outbreak Associated,60 to 69 Years,Moss Park,M5A,"Outbreaks, Congregate Settings",CONFIRMED,2020-04-23,2020-05-06,MALE,RESOLVED,No,No,No,Yes,No,No
5,263,174703,278,Sporadic,60 to 69 Years,Niagara,M5V,Community,CONFIRMED,2020-03-16,2020-03-22,MALE,FATAL,No,No,No,Yes,Yes,Yes
6,270,174710,286,Outbreak Associated,60 to 69 Years,Bedford Park-Nortown,M6A,"Outbreaks, Healthcare Institutions",CONFIRMED,2020-03-19,2020-03-19,MALE,RESOLVED,No,No,No,Yes,No,No
7,530,174970,562,Sporadic,60 to 69 Years,Bendale,M1P,Travel,CONFIRMED,2020-03-24,2020-03-26,MALE,RESOLVED,No,No,No,Yes,Yes,Yes
8,552,174992,586,Sporadic,60 to 69 Years,Mimico (includes Humber Bay Shores),M8V,Community,CONFIRMED,2020-03-21,2020-03-27,MALE,RESOLVED,No,No,No,Yes,Yes,Yes
9,597,175037,633,Sporadic,60 to 69 Years,Englemount-Lawrence,M6A,Community,CONFIRMED,2020-03-23,2020-03-27,MALE,RESOLVED,No,No,No,Yes,No,No


Use the groupby clause to group your entries in the data frame by Gender and query the **total count** of cases for each Gender. 


In [6]:
pd.read_sql(
    "SELECT [Client Gender],COUNT([Client Gender]) \
    FROM covid \
    GROUP BY [Client Gender]"
    ,con = conn
)

,Client Gender,COUNT(
0,FEMALE,47067
1,MALE,45389
2,NON-BINARY,1
3,OTHER,19
4,TRANSGENDER,11
5,UNKNOWN,698


#Joins

Create two tables named Address and Number as:


```
+----+-------+--------------+
| Id | Name  | Location     | 
+----+-------+--------------+
| 1  | Joe   | 1370 Way St. |
| 2  | Henry | 34 Apple Dr. |
| 3  | Sam   | 60 Moon Rd.  |
| 4  | Max   | 95 Main St.  |
+----+-------+--------------+
```



```
+----+-------+--------------+
| Id | Name  | Phone Number | 
+----+-------+--------------+
| 1  | Joe   | 1234567890   |
| 5  | Candy | 1230987654   |
| 7  | Bob   | 1235476980   |
| 4  | Max   | 1239078564   |
+----+-------+--------------+
```

Use the appropriate Join to Get an output as:



```
+----+-------+--------------+--------------+
| Id | Name  | Location     | Number       |
+----+-------+--------------+--------------+
| 1  | Joe   | 1370 Way St. | 1234567890   |
| 2  | Henry | 34 Apple Dr. |              |
| 3  | Sam   | 60 Moon Rd.  |              |
| 4  | Max   | 95 Main St.  | 1239078564   |
+----+-------+--------------+--------------+
```




In [7]:
t1 = pd.DataFrame(([
         [1,"Joe","1370 Way St."],
         [2,'Henry',"34 Apple Dr."],
         [3,"Sam", "60 Moon Rd.",],
         [4,"Max", "95 Main St." ]]),
         columns = ["Id", "Name", "Location"]
         )
t2 = pd.DataFrame(([
         [1,"Joe", 1234567890],
         [5,"Candy",1230987654],
         [7,"Bob",1235476980],
         [4,"Max",1239078564]]),
         columns = ["Id", "Name", "Phone Number"])
t1.to_sql("Address",index = False,con=conn)
t2.to_sql("Number", index = False,con=conn)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [8]:
pd.read_sql(
    "SELECT * \
    FROM Address \
    "
    ,con = conn)

,Id,Name,Location
0,1,Joe,1370 Way St.
1,2,Henry,34 Apple Dr.
2,3,Sam,60 Moon Rd.
3,4,Max,95 Main St.


In [9]:
pd.read_sql(
    "SELECT a.*, IFNULL(n.[Phone Number],'') \
    FROM Address a \
    LEFT JOIN Number n on n.Id = a.Id "
    ,con = conn)

,Id,Name,Location,IFNULL(n.
0,1,Joe,1370 Way St.,1234567890
1,2,Henry,34 Apple Dr.,
2,3,Sam,60 Moon Rd.,
3,4,Max,95 Main St.,1239078564


#Technical Questions

**NOTE** You are going to have to create the initial table or Schema first for the following questions.

##Duplicate Emails



Write a SQL query to find all duplicate emails in a table named Person.



```
+----+---------+
| Id | Email   |
+----+---------+
| 1  | a@b.com |
| 2  | c@d.com |
| 3  | a@b.com |
+----+---------+
```

For example, your query should return the following for the above table:

```
+---------+
| Email   |
+---------+
| a@b.com |
+---------+
```





In [10]:
df = pd.DataFrame(([[1,"a@b.com"],[2,"c@d.com"],[3,"a@b.com"]]), columns=["Id", "Email"])

df.to_sql("Email",index=False,con = conn)

In [11]:
pd.read_sql(
    "SELECT Email \
    FROM Email \
    GROUP BY Email \
    HAVING count(email) > 1"
    ,con = conn)

,Email
0,a@b.com


##Big Countries

There is a table World



```
+-----------------+------------+------------+--------------+---------------+
| name            | continent  | area       | population   | gdp           |
+-----------------+------------+------------+--------------+---------------+
| Afghanistan     | Asia       | 652230     | 25500100     | 20343000      |
| Albania         | Europe     | 28748      | 2831741      | 12960000      |
| Algeria         | Africa     | 2381741    | 37100000     | 188681000     |
| Andorra         | Europe     | 468        | 78115        | 3712000       |
| Angola          | Africa     | 1246700    | 20609294     | 100990000     |
+-----------------+------------+------------+--------------+---------------+
```
A country is big if it has an area of bigger than 3 million square km or a population of more than 25 million.

Write a SQL solution to output big countries' name, population and area.

For example, according to the above table, we should output:


```
+--------------+-------------+--------------+
| name         | population  | area         |
+--------------+-------------+--------------+
| Afghanistan  | 25500100    | 652230       |
| Algeria      | 37100000    | 2381741      |
+--------------+-------------+--------------+
```




In [12]:
df  = pd.DataFrame(([
                     ["Afghanistan","Asia",652230,25500100,20343000],
                     ["Albania","Europe",28748,2831741,12960000],
                     ['Algeria',"Africa",2381741,37100000,188681000],
                     ["Andorra","Europe",468,78115,3712000],
                     ["Angola","Africa",1246700,20609294,100990000]
                     ]),
                    columns = ["name","continent","area","population","gdp"]
                   )
df.to_sql("World", con = conn, index = False)

In [15]:
pd.read_sql(
    "SELECT name, population, area \
    FROM World \
    WHERE area > 3000000 \
    OR population > 25000000 "
    ,con = conn
)

,name,population,area
0,Afghanistan,25500100,652230
1,Algeria,37100000,2381741


#Not Boring Movies

X city opened a new cinema, many people would like to go to this cinema. The cinema also gives out a poster indicating the movies’ ratings and descriptions.
Please write a SQL query to output movies with an odd numbered ID and a description that is not 'boring'. Order the result by rating.

For example, table cinema:

```
+---------+-----------+--------------+-----------+
|   id    | movie     |  description |  rating   |
+---------+-----------+--------------+-----------+
|   1     | War       |   great 3D   |   8.9     |
|   2     | Science   |   fiction    |   8.5     |
|   3     | irish     |   boring     |   6.2     |
|   4     | Ice song  |   Fantacy    |   8.6     |
|   5     | House card|   Interesting|   9.1     |
+---------+-----------+--------------+-----------+
```
For the example above, the output should be:

```
+---------+-----------+--------------+-----------+
|   id    | movie     |  description |  rating   |
+---------+-----------+--------------+-----------+
|   5     | House card|   Interesting|   9.1     |
|   1     | War       |   great 3D   |   8.9     |
+---------+-----------+--------------+-----------+
```




In [17]:
df  = pd.DataFrame(([
                     [1,"War","great 3d",8.9],
                     [2,"Science","fiction",8.5],
                     [3,'Irish',"boring",6.2],
                     [4,"Ice song","Fantasy",8.6],
                     [5,"House card","Interesting",9.1]
                     ]),
                    columns = ["id","movie","description","rating"]
                   )
df.to_sql("cinema", con = conn, index = False)

In [21]:
pd.read_sql(
    "SELECT * \
    FROM cinema \
    WHERE id%2 = 1 \
    AND description != 'boring' \
    ORDER BY rating DESC"
    , con = conn
)

,id,movie,description,rating
0,5,House card,Interesting,9.1
1,1,War,great 3d,8.9


#BONUS

##Department Highest Salary

The Employee table holds all employees. Every employee has an Id, a salary, and there is also a column for the department Id.


```
+----+-------+--------+--------------+
| Id | Name  | Salary | DepartmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 70000  | 1            |
| 2  | Jim   | 90000  | 1            |
| 3  | Henry | 80000  | 2            |
| 4  | Sam   | 60000  | 2            |
| 5  | Max   | 90000  | 1            |
+----+-------+--------+--------------+
```

The Department table holds all departments of the company.



```
+----+----------+
| Id | Name     |
+----+----------+
| 1  | IT       |
| 2  | Sales    |
+----+----------+
```
Write a SQL query to find employees who have the highest salary in each of the departments. For the above tables, your SQL query should return the following rows (order of rows does not matter).

```
+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Max      | 90000  |
| IT         | Jim      | 90000  |
| Sales      | Henry    | 80000  |
+------------+----------+--------+
```

Explanation:

Max and Jim both have the highest salary in the IT department and Henry has the highest salary in the Sales department.

In [26]:
t1 = pd.DataFrame(([[1,"Joe",70000,1],
                    [1,"Jim",90000,1],
                    [1,"Henry",80000,2],
                    [1,"Sam",60000,2],
                    [1,"Max",90000,1]
                    ]),
                    columns = ["Id", "Name","Salary","DepartmentId"]
                    )

t2 = pd.DataFrame(([[1,"IT"],
                    [2,"Sales"]
                    ]),
                    columns = ["Id", "Name"]
                    )
t1.to_sql("Employee",con = conn, index = False)
t2.to_sql("Department",con = conn, index = False)

In [58]:
pd.read_sql(
    "SELECT d.Name as Department, e.Name, e.Salary \
    FROM employee e \
    JOIN Department d on d.Id = e.DepartmentId \
    ORDER BY e.salary desc \
    LIMIT 3",
    con = conn
)

,Department,Name,Salary
0,IT,Jim,90000
1,IT,Max,90000
2,Sales,Henry,80000
